## 11-2. 다중 회귀 분석(Multiple Regression Analysis)

- 목표 변수(Y, Target)값이 하나이고, 설명변수(Input, X)값이 여러개 있을 때 수행하는 회귀분석
- 여기서 회귀계수는(기울기) 최소제곱법(Least Square Metohd)계산
- 다중 회귀 분석에서는 **다중공선성(Multi-Collinearity)** 을 고려
- **다중공선성(Multi-Collinearity)** : 설명변수 X간 상관관계가 높아 발생하는 문제로, 설명변수 간 관계가 강한 변수들에 의해 수식이 구성될 때, 통제할 수 없는 오차
- **분산팽창지수(Variance Inflation Factor, VIF)** 을 이용하여, 다중공선성의 문제가 발생하는 변수를 파악, 처리
- 변수를 선택해 나가며 모델을 개선하는 작업을 수행

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import scipy.stats as stats

In [2]:
df = pd.read_csv('data/12_Data.csv')
print(df.shape)
df.head(2)

(569, 32)


,Image ID,Diagnosis,Mean Radius,Mean Perimeter,Mean Area,Mean Texture,Mean Smoothness,Mean Compactness,Mean Concavity,Mean Concave Points,...,SE Radius,SE Perimeter,SE Area,SE Texture,SE Smoothness,SE Compactness,SE Concavity,SE Concave Points,SE Symmetry,SE Fractal Dim
0,842302,M,17.99,122.8,1001.0,10.38,0.12,0.27760,0.3001,0.1471,...,1.0950,8.589,153.40,0.9053,0.0064,0.0490,0.0537,0.0159,0.0300,0.0062
1,842517,M,20.57,132.9,1326.0,17.77,0.08,0.07864,0.0869,0.0702,...,0.5435,3.398,74.08,0.7339,0.0052,0.0131,0.0186,0.0134,0.0139,0.0035


In [6]:
df.columns

Index(['Image ID', 'Diagnosis', 'Mean Radius', 'Mean Perimeter', 'Mean Area',
       'Mean Texture', 'Mean Smoothness', 'Mean Compactness', 'Mean Concavity',
       'Mean Concave Points', 'Mean Symmetry', 'Mean Fractal Dim',
       'Max Radius', 'Max Perimeter', 'Max Area', 'Max Texture',
       'Max Smoothness', 'Max Compactness', 'Max Concavity',
       'Max Concave Points', 'Max Symmetry', 'Max Fractal Dim', 'SE Radius',
       'SE Perimeter', 'SE Area', 'SE Texture', 'SE Smoothness',
       'SE Compactness', 'SE Concavity', 'SE Concave Points', 'SE Symmetry',
       'SE Fractal Dim'],
      dtype='object')

In [12]:
# 다중 회귀 모델 구성
# Target Y값과 Input X값 설정
x = df[['Mean Perimeter', 'Mean Area',
       'Mean Texture', 'Mean Smoothness', 'Mean Compactness', 'Mean Concavity',
       'Mean Concave Points', 'Mean Symmetry', 'Mean Fractal Dim']]
y = df['Mean Radius']

In [13]:
import statsmodels.api as sm

# 상수항을 계산하기 위한 Column추가
x_const = sm.add_constant(x)

In [14]:
model = sm.OLS(y, x_const) # 모델을 구성
result = model.fit() # 구성된 모델을 최소제곱법으로 연산
result.summary() # 연산 결과를 확인

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Mean Radius   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 9.973e+04
Date:                Wed, 16 Apr 2025   Prob (F-statistic):               0.00
Time:                        09:25:34   Log-Likelihood:                 576.57
No. Observations:                 569   AIC:                            -1133.
Df Residuals:                     559   BIC:                            -1090.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.1111      0.134      0.832      0.406      -0.151       0.374
Mean Perimeter          0.1567      0.001    117.406      0.000       0.154       0.159
Mean Area              -0.0003   7.84e-05     -3.546      0.000      -0.000      -0.000
Mean Texture            0.0003      0.001      0.283      0.778      -0.002       0.002
Mean Smoothness         1.1996      0.430      2.792      0.005       0.356       2.044
Mean Compactness       -4.8143      0.265    -18.155      0.000      -5.335      -4.293
Mean Concavity         -0.7665      0.156     -4.909      0.000      -1.073      -0.460
Mean Concave Points    -0.2747      0.445     -0.618      0.537      -1.148       0.599
Mean Symmetry           0.2399      0.180      1.330      0.184      -0.114       0.594
Mean Fractal Dim        3.2373      1.335      2.425      0.016       0.615       5.860
==============================================================================
Omnibus:                      116.768   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2025.407
Skew:                          -0.335   Prob(JB):                         0.00
Kurtosis:                      12.218   Cond. No.                     2.73e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.73e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
# 분산 팽창 지수를 계산
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF값을 하나의 데이터 테이블로 계산
data_VIF = pd.DataFrame()
data_VIF['x'] = x_const.columns
data_VIF['vif'] = [variance_inflation_factor(x_const.values, x) for x in range(x_const.shape[1])]
data_VIF
# vif값이 1에 가까우면 다른 독립변수들과 상관관계가 낮음
# vif값이 1보다 크면 다른 독립변수들과 상관관계가 존재
# vif값이 5보다 크면 해당 독립변수가 다른 독립변수들과 상당한 다중 공선성을 가질 가능성이 있음


,x,vif
0,const,1293.292556
1,Mean Perimeter,76.100504
2,Mean Area,55.015152
3,Mean Texture,1.190713
4,Mean Smoothness,2.805506
5,Mean Compactness,14.185253
6,Mean Concavity,11.205207
7,Mean Concave Points,21.536872
8,Mean Symmetry,1.767032
9,Mean Fractal Dim,6.426153


- 다중공선성 문제를 해결하기 위한 변수 선택법
  - 1. 후진제거법(Backward Elimination): 모든 X를 포함하여 모델을 먼저 구성한 뒤, 가장 영향력이 적은 X들을 제거해 나가며 모델을 구성
   
  - 2. 전진선택법(Forward Selection): 설명력이 가장 큰 변수부터 시작하여, 순차적으로 X를 추가하며 회귀식에 기여하는 정도에 따라 변수를 선택
   
  - 3. 단계적 선택법(Stepwise): 후진제거법과 전진선택법의 단점을 보완해 여러단계로 모델을 구성해 나가며 가장 성능이 높은 모델을 찾는 방법

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

In [18]:
# 모델 구성
model = LinearRegression()

# 전체 변수 중, 5개 변수가 남을 때 까지 변수를 제거
rfe_model = RFE(estimator = model, n_features_to_select = 5).fit(x, y)

In [24]:
# 선택된 변수
select_columns = x.columns[rfe_model.support_]
select_columns

Index(['Mean Smoothness', 'Mean Compactness', 'Mean Concave Points',
       'Mean Symmetry', 'Mean Fractal Dim'],
      dtype='object')

In [25]:
x_const_rfe = sm.add_constant(x_const[select_columns])
model = sm.OLS(y, x_const_rfe)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Mean Radius   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     977.3
Date:                Wed, 16 Apr 2025   Prob (F-statistic):          9.36e-275
Time:                        10:06:21   Log-Likelihood:                -877.77
No. Observations:                 569   AIC:                             1768.
Df Residuals:                     563   BIC:                             1794.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  27.4806      0.619     44.399      0.000      26.265      28.696
Mean Smoothness       -25.8355      4.950     -5.220      0.000     -35.558     -16.113
Mean Compactness       11.2076      2.633      4.256      0.000       6.036      16.379
Mean Concave Points    76.9620      3.117     24.692      0.000      70.840      83.084
Mean Symmetry          -9.0868      2.280     -3.986      0.000     -13.564      -4.609
Mean Fractal Dim     -225.4089     12.417    -18.153      0.000    -249.799    -201.019
==============================================================================
Omnibus:                        6.681   Durbin-Watson:                   1.923
Prob(Omnibus):                  0.035   Jarque-Bera (JB):                7.937
Skew:                           0.143   Prob(JB):                       0.0189
Kurtosis:                       3.503   Cond. No.                         280.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
# 1. Tube Furnace CP(소입로 가스 침탄량)과 Corrleation이 높은 상위 5개의 항목을 확인하시오.
df = pd.read_csv('../hanro_test/0415/08_Data.csv')
print(df.shape)
df.head(2)

(93043, 17)


,Unnamed: 0,Code_Num,Datetime,Process_Type,ST,Tube Furnace CP,Tube Furnace1 OP,Tube Furnace1 Temp,Tube Furnace2 OP,Tube Furnace2 Temp,Tube Furnace3 OP,Tube Furnace3 Temp,Tube Furnace4 OP,Tube Furnace4 Temp,Z1-OP1,Z1-OP2,Z1-Temp
0,0,GroupA,2022-01-03 11:22,OP-A,NaN,0.450497,73.527404,NaN,59.989422,868.759544,52.249481,NaN,72.134908,NaN,74.255730,76.879748,NaN
1,1,GroupA,2022-01-03 11:22,OP-A,NaN,0.457370,79.149174,878.116929,62.027232,883.821264,50.617109,890.74076,71.245235,894.430606,66.323571,71.297720,NaN


In [37]:
# 연속형 * 연속형 -> 연속형 자료가 정규분포를 따르는가?
# 각 연속형 자료들이 정규성을 따르는지 확인
numeric_list = df.describe().columns # describe함수가 연속형 자료들만 요약 통계량을 계싼하는 점을 활용하여 함수의 결과의 column명을 추출
df_num = df[numeric_list[1:]] 
# 연속형 자료들로만 구성된 데이터프레임의 각 column을 한줄 씩 가져와 정규성 검정을 수행
normal_list = []
nonnormal_list = []
for i in numeric_list[1:]:
    stats_result, pvalue_result = stats.normaltest(df_num[i].dropna())
    if pvalue_result < 0.05: # 대립가설 참 : 정규분포를 따르지 않는 데이터의 항목 명을 리스트에 추가
        nonnormal_list.append(i)
    else:
        normal_list.append(i)

In [38]:
# 정규분포를 따르는 연속형 자료들의 상관계수를 확인
df[normal_list].corr()

,Z1-OP1,Z1-OP2
Z1-OP1,1.000000,-0.002423
Z1-OP2,-0.002423,1.000000


In [40]:
# 정규분포를 따르지 않는 연속형 자료들의 상관계수를 확인
df[nonnormal_list].corr(method = 'spearman')['Tube Furnace CP']

ST                   -0.008257
Tube Furnace CP       1.000000
Tube Furnace1 OP     -0.017604
Tube Furnace1 Temp    0.005490
Tube Furnace2 OP      0.006694
Tube Furnace2 Temp    0.007430
Tube Furnace3 OP      0.013718
Tube Furnace3 Temp   -0.001855
Tube Furnace4 OP     -0.012107
Tube Furnace4 Temp   -0.003243
Z1-Temp              -0.012176
Name: Tube Furnace CP, dtype: float64

In [ ]:
#2. 앞서 확인한 5개 항목을 X로, Tube Furnace CP을 Y로 선언하여, 전통 통계에서 사용하는 회귀 기법을 이용해 회귀 분석을 실시하시오.


In [ ]:
#3. 회귀 분석 결과에 대해 아래의 사항을 진단하시오.
#    - 아래 진단사항에 가설을 수립하고, 회귀 분석 결과를 이용해 해석하시오.
#    - 3-1. 회귀선이 기울기가 존재하고 경회귀 선이 특정 경향성을 따르는가?
#    - 3-2. 각 회귀 계수들의 값이 회귀식에 대해 유의미한가?
#    - 3-3. 회귀 모델의 오차 항이 정규 분포를 따르는가?


In [ ]:
#4. 회귀식을 이용해, 5개의 값이 각각 순서대로 50, 80, 100, 75, 200 값이 왔을 때, Y값(Tube Furnace CP(소입로 가스 침탄량)을 계산하시오.


In [ ]:
#5. Z1-OP1(공정건조온도 출력량) 평균에 대해 Z1-OP1 표준편차값이 3배이상 벗어나는 값을 '이상' 나머지를 '정상'으로 'Label'이라는 변수를 생성하고, Code_Num(공정배치그룹)과 'Label' 값이 서로 독립인지 가설 검정을 수행하시오.
#    - 가설 및 결론까지 작성